In [2]:
import h5py
import json
import pandas as pd
import numpy as np
import torch
import os
import pickle
import ast
from data.dataset import VQADataset
import linecache
import random
from datetime import datetime

In [5]:
% timeit pd.read_pickle('./data/processed/train_dataset.gzip')
% timeit pd.read_pickle('./data/processed/val_dataset.gzip')
% timeit pd.read_pickle('./data/processed/test_dataset.gzip')
% timeit pd.read_pickle('./data/processed/dummy_dataset.gzip')

595 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
76.8 ms ± 685 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
22.9 ms ± 584 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2 ms ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
df = pd.read_pickle('./data/processed/train_dataset.gzip')
print(type(df.loc[0, 'visual_features']))
df.head()

<class 'numpy.ndarray'>


,question_id,image_id,question,answer,visual_features
0,228478002,228478,What English meal is this likely for ?,brunch,"[0.21652411, 0.4433445, 1.0575547, 0.351903, 0..."
1,111756005,111756,What color is his uniform ?,blue,"[0.88861424, 1.5218604, 0.41224426, 1.0292011,..."
2,376241080,376241,Which girl is wearing glasses ?,right,"[0.15327102, 0.80850273, 0.49285752, 0.1413253..."
3,434045001,434045,What is the person doing ?,sunbathing,"[3.634467, 0.121703975, 0.9868309, 0.114410535..."
4,167330014,167330,How does the weather appear in this photo ?,sunny,"[1.5159967, 0.5695442, 0.51491714, 0.49372593,..."


In [31]:
df = pd.read_pickle('./data/processed/dummy_dataset.gzip')
len([len(answer.split()) for answer in df['answer']]) # only have one-word answers. Good.

128

In [33]:
with open('./data/processed/label2idx.pkl', 'rb') as f:
    labels2idx = pickle.load(f)
print([labels2idx.get(answer, "NASOOOL") for answer in df['answer']])

[12, 25, 74, 441, 10, 79, 16, 162, 19, 56, 29, 183, 70, 1, 1, 629, 93, 1, 1, 1, 38, 102, 106, 240, 1, 1, 314, 2, 1, 2, 1, 264, 315, 15, 12, 265, 1, 2, 1, 64, 163, 12, 16, 1, 2, 1, 796, 13, 240, 56, 36, 3, 22, 524, 16, 1, 5, 94, 1, 1, 1, 1, 3, 210, 22, 17, 36, 1, 37, 58, 195, 224, 1, 283, 136, 5, 1, 5, 110, 357, 1, 1, 3, 49, 15, 224, 10, 630, 442, 266, 6, 15, 316, 77, 79, 2, 107, 1, 10, 1, 93, 2, 1, 184, 3, 8, 32, 11, 10, 39, 2, 1, 1, 1, 443, 1, 1, 1, 24, 3, 10, 525, 2, 38, 11, 1, 5, 185]


In [36]:
df = pd.read_pickle('./data/processed/dummy_dataset.gzip')
print([labels2idx.get(answer, "NASOOOL") for answer in df['answer']])

[12, 25, 74, 441, 10, 79, 16, 162, 19, 56, 29, 183, 70, 1, 1, 629, 93, 1, 1, 1, 38, 102, 106, 240, 1, 1, 314, 2, 1, 2, 1, 264, 315, 15, 12, 265, 1, 2, 1, 64, 163, 12, 16, 1, 2, 1, 796, 13, 240, 56, 36, 3, 22, 524, 16, 1, 5, 94, 1, 1, 1, 1, 3, 210, 22, 17, 36, 1, 37, 58, 195, 224, 1, 283, 136, 5, 1, 5, 110, 357, 1, 1, 3, 49, 15, 224, 10, 630, 442, 266, 6, 15, 316, 77, 79, 2, 107, 1, 10, 1, 93, 2, 1, 184, 3, 8, 32, 11, 10, 39, 2, 1, 1, 1, 443, 1, 1, 1, 24, 3, 10, 525, 2, 38, 11, 1, 5, 185]


In [3]:
dataset = VQADataset("val", True, 20)

VQADataset initialised in 0:00:00.095646


In [5]:
for idx in range(10):
    question, vl_feat, answer = dataset[idx]
    print(dataset.convert_question_to_string(question), dataset.convert_answer_to_string(answer))

<bos> what is standing on the computer <eos> cat
<bos> what are the animals standing on <eos> sand
<bos> is the sky cloudy <eos> yes
<bos> are two men in the background on the left wearing hawaiian shirts <eos> yes
<bos> where is the person located <eos> mountain
<bos> is it a sunny day <eos> yes
<bos> how many zebras <eos> 4
<bos> is this a dining car <eos> yes
<bos> what is the man in the white shirt riding <eos> skateboard
<bos> how many neckties are on display <eos> 3


In [6]:
len(dataset)

6392